In [1]:
import pandas as pd
import sys
read_path = "~/Dropbox (ASU)/Google Analytics/all/"


In [2]:
# Set path and get helpers
sys.path.append("/Users/saaryalov/GoogleAnalyticsKaggle/")


In [3]:
%%time
train = pd.read_csv(read_path+"extracted_fields_train.gz", dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)
test = pd.read_csv(read_path+"extracted_fields_test.gz", dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)

CPU times: user 12.3 s, sys: 773 ms, total: 13.1 s
Wall time: 18 s


In [4]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
       'trafficSource.adContent', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [5]:
train["totals.transactionRevenue"].fillna(0,inplace=True)


In [6]:
# Change to DateTime
for df in [train, test]:
    df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['sess_date_dow'] = df['date'].dt.dayofweek
    df['sess_date_hours'] = df['date'].dt.hour
    df['sess_date_dom'] = df['date'].dt.day
    
    del df['date']
    del df['visitStartTime']


In [7]:
print(f"Notice that {train.shape} is different then used on kernels, it is because in most kernels low variance features were dropped")

Notice that (903653, 31) is different then used on kernels, it is because in most kernels low variance features were dropped


# Device FE

In [8]:
from helpers import *
device_ls = ['device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem']
for each in device_ls:
    result = sort_by_rev(train,each,kind="sum")
    sm = sum(result["totals.transactionRevenue"]>0)
    print(f"{each} has levels >0 in : {sm} with shape : {result.shape}")


device.browser has levels >0 in : 9 with shape : (54, 3)
device.deviceCategory has levels >0 in : 3 with shape : (3, 3)
device.isMobile has levels >0 in : 2 with shape : (2, 3)
device.operatingSystem has levels >0 in : 7 with shape : (20, 3)


In [9]:
result = sort_by_rev(train,'device.isMobile',kind="sum")
result.head(5)

,device.isMobile,Frequency,totals.transactionRevenue
0,0.0,0.735382,1.480832e+12
1,1.0,0.264618,5.923961e+10


In [10]:
result = sort_by_rev(train,'device.deviceCategory',kind="sum")
print(result.head(5))

# device.isMobile is redundent, so remove:
del train['device.isMobile']

  device.deviceCategory  Frequency  totals.transactionRevenue
0               desktop   0.735325               1.480864e+12
1                mobile   0.230979               4.978581e+10
2                tablet   0.033695               9.421340e+09


In [12]:
from helper.CategoricalLevelSelector import CategoricalLevelSelection
cls = CategoricalLevelSelection() 
cls.fit(train,key="fullVisitorId",cols_to_ignore=["sessionId"],verbose=True)
cols = ['channelGrouping', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber','device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       ]
fitted = cls.transform(train[cols],verbose=True)

Working on channelGrouping
Working on device.browser
Working on device.deviceCategory
Working on device.operatingSystem
Working on geoNetwork.city
Working on geoNetwork.continent
Working on geoNetwork.country
Working on geoNetwork.metro
Working on geoNetwork.networkDomain
Working on geoNetwork.region
Working on geoNetwork.subContinent
Working on trafficSource.adContent
Working on trafficSource.campaign
Working on trafficSource.keyword
Working on trafficSource.medium
Working on trafficSource.referralPath
Working on trafficSource.source
changing channelGrouping
changing device.browser
changing device.deviceCategory
changing device.operatingSystem
changing geoNetwork.city
